In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import tqdm

## Functions

In [ ]:
def parent_replication(population, fitness, size):
    """Select parents via replication method.

    Parameters
    ----------
    population : np.array
        The current population.
    fitness : np.array
        The fitness of each member in the population.
    size : int
        The population size.

    Returns
    -------
    parents : np.array
        The parents sampled from the fitness probability distribution.
    """
    # Approximate probability distribution from inverse of fitness (low fitness has high probability)
    fitness_inv = 1 / fitness
    fitness_pdf = fitness_inv / fitness_inv.sum()
    # Randomly sample indices from fitness probability distribution
    rand_inds = np.random.choice(np.arange(size), size=size, p=fitness_pdf)
    # Select parents from random indices
    parents = population[rand_inds]
        
    return parents

In [ ]:
def parent_tournament(population, fitness, size, n_params, n_cands_per_child):
    """Select parents via tournament method.

    Parameters
    ----------
    population : np.array
        The current population.
    fitness : np.array
        The fitness of each member in the population.
    size : int
        The population size.
    n_params : int
        The number of parameters in the population.
    n_cands_per_child : int
        The number of parent candidates to produce a child.

    Returns
    -------
    parents : np.array
        The parents with the best fitness from each tournament.
    """
    # Randomly choose parent candidates
    n_cands = size * n_cands_per_child
    rand_inds = np.random.choice(np.arange(size), size=n_cands)
    # Select parent and fitness candidates for tournament
    parent_cands = population[rand_inds].reshape(size, n_cands_per_child, n_params)
    fitness_cands = fitness[rand_inds].reshape(size, n_cands_per_child)
    # Select parents with minimum fitness in each group of n_cands_per_child
    min_inds = np.argmin(fitness_cands, axis=1)
    parents = parent_cands[np.arange(size), min_inds]
    
    return parents

In [ ]:
def crossover(parents, crossover_rate):
    """Produce children by crossing over parents.

    Parameters
    ----------
    parents : np.array
        The parents to crossover.
    crossover_rate : float
        The crossover rate.

    Returns
    -------
    children : np.array
        The children produced by crossing over parents.
    """
    # Find crossover index
    cross_ind = int(crossover_rate * parents.shape[1])
    # Slice parents at crossover index
    children1 = parents[:, :, :cross_ind]
    # Crossover genes at crossover index
    children2 = np.flip(parents[:, :, cross_ind:], axis=1)
    # Produce children by concatenating genes
    children = np.concatenate((children1, children2), axis=2)
    
    return children

In [ ]:
def mutation(children, mutation_rate):
    """Randomly mutate children.

    Parameters
    ----------
    children : np.array
        The children produced by crossing over parents.
    mutation_rate : float
        The mutation rate.

    Returns
    -------
    children : np.array
        The children after mutating.
    """
    # Generate mutation mask
    mutation_mask = np.random.random(children.shape) <= mutation_rate
    # Mutate children by adding noise sampled from a standard normal distribution
    children[mutation_mask] += np.random.standard_normal(mutation_mask.sum())
    
    return children

In [ ]:
def genetic_algorithm(
    x, 
    y, 
    generations=100, 
    population=None, 
    size=800, 
    select_parents='tournament', 
    n_params=2, 
    n_cands_per_child=8, 
    crossover_rate=0.5, 
    mutation_rate=0.05
):

    """Execute the genetic algorithm to optimize parameters.

    If not provided, an random population is set. For each generation:
    1. The popolation and fitness are evaluated and recorded.
    2. Parents are selected via replication or tournament.
    3. Parents are shuffled and paired for reproduction.
    4. Parents produce children via crossing over.
    5. New population is set by children after mutation.

    Parameters
    ----------
    x : np.array
        Inputs of a function.
    y : np.array
        Outputs of a function.
    generations : int, defult=1000
        The number of generations to simulate.
    population : np.array, default=None
        The starting population. If None, sample population from N(0,1).
    size : int
        The population size.
    select_parents : str, default='tournament'
        Method for selecting parents. Only valid arguments are 'replication' and 'tournament'.
    n_params : int, default=2
        The number of parameters in the population.
    n_cands_per_child : int, default=8
        The number of parent candidates to produce a child.
    crossover_rate : float, default=0.5
        The crossover rate.
    mutation_rate : float, default=0.05
        The mutation rate.

    Returns
    -------
    population : np.array
        The final generation's population.
    fitness_metrics : np.array
        The mean, standard deviation, and minimum fitness of each generation.
    """
    
    # Make initial random population if necessary
    if population == None:
        population = np.random.standard_normal(size=(size, n_params))
    
    # Initiate metrics
    metrics = np.zeros((generations, 7))
    
    for i in tqdm.tqdm(range(generations)):
        # Find fitness of each sample
        fit = population[:, 0:1] @ x.reshape(1, -1) + population[:, 1:]
        fitness = np.sum((fit - y) ** 2, axis=1)
        
        # Record metrics
        metrics[i, 0:3] = [fitness.mean(), fitness.std(), fitness.min()]
        metrics[i, 3:5] = np.median(population, axis=0)
        metrics[i, 5:7] = scipy.stats.median_abs_deviation(population, axis=0)
        
        # Select parents
        if select_parents == 'replication':
            parents = parent_replication(population, fitness, size)
        elif select_parents == 'tournament':
            parents = parent_tournament(population, fitness, size, n_params, n_cands_per_child)
        else:
            raise ValueError(f'{select_parents} is invalid. Use "replication" or "tournament".')

        # Pair up shuffled parents to reproduce
        np.random.shuffle(parents)
        parents = parents.reshape(size // 2, 2, n_params)
        
        # Produce children by crossing over parents
        children = crossover(parents, crossover_rate)
        children = children.reshape(size, n_params)
        
        # Produce new population by mutating children
        population = mutation(children, mutation_rate)

    # Convert metrics to dataframe
    columns = ['fitness_mean', 'fitness_std', 'fitness_min', 'w_median', 'b_median', 'w_mad', 'b_mad']
    metrics = pd.DataFrame(metrics, columns=columns)
    return population, metrics

## Simulate data

In [ ]:
width = 20
x = np.arange(-5, 5, 0.1)

w_true = (np.random.rand() - 0.5) * width
b_true = (np.random.rand() - 0.5) * width
y_true = w_true * x + b_true

noise = np.sqrt(width) * np.random.randn(x.shape[0])
y = y_true + noise

In [ ]:
plt.title(f'{w_true:.3f}x+{b_true:.3f}')
plt.grid()
plt.scatter(x, y)
plt.xlabel('X')
plt.ylabel('Y')

## Fit and evaluate

In [ ]:
population, metrics = genetic_algorithm(x, y)

In [ ]:
w_median, b_median = np.median(population, axis=0)
print (f'True params: weight={w_true}, bias={b_true}')
print (f'Evolved params: weight={w_median}, bias={b_median}')

In [ ]:
fig, axs = plt.subplots(1,2,figsize=[10,5])

axs[0].set_title('Fitness Metrics')
axs[0].grid()
axs[0].plot(metrics['fitness_mean'], label='mean')
axs[0].plot(metrics['fitness_std'], label='std')
axs[0].plot(metrics['fitness_min'], label='min')
axs[0].set_xlabel('Generation')
axs[0].set_ylabel('Fitness')
axs[0].set_xscale('log')
axs[0].set_yscale('log')
axs[0].legend()

axs[1].set_title('Parameter Metrics')
axs[1].grid()
axs[1].errorbar(metrics.index, metrics['w_median'], metrics['w_mad'], label='weight')
axs[1].errorbar(metrics.index, metrics['b_median'], metrics['b_mad'], label='bias')
axs[1].hlines(w_true, 0, metrics.index.max(), color='C0', ls='--', label='true weight')
axs[1].hlines(b_true, 0, metrics.index.max(), color='C1', ls='--', label='true bias')
axs[1].set_xlabel('Generation')
axs[1].set_ylabel('Parameters')
axs[1].set_xscale('log')
axs[1].legend(ncol=2)

In [ ]:
y_pred_generations = metrics['w_median'].values.reshape(-1, 1) @ x.reshape(1, -1) + metrics['b_median'].values.reshape(-1, 1)
y_pred = w_median * x + b_median

plt.title('GA Fits')
plt.grid()
plt.scatter(x, y, label=f'true: w={w_true:.3f}, b={b_true:.3f}')
for y_pred_gen_i in y_pred_generations:
    plt.plot(x, y_pred_gen_i, color='C1', alpha=0.1)
plt.plot(x, y_pred, color='C1', label=f'ga: w={w_median:.3f}, b={b_median:.3f}')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()

## Compare with least squares

In [ ]:
z = np.polyfit(x, y, deg=1)
p = np.poly1d(z)
fit = p(x)

In [ ]:
plt.title('Least Squares vs GA')
plt.grid()
plt.scatter(x, y, label=f'true: w={w_true:.3f}, b={b_true:.3f}')
plt.plot(x, y_pred, color='C1', 
         label=f'ga: w={w_median:.3f}, b={b_median:.3f}')
plt.legend()
plt.plot(x, fit, '--', color='C2',
         label=f'polyfit: w={z[0]:.3f}, b={z[1]:.3f}')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()

In [ ]:
fig, axs = plt.subplots(1,2,figsize=[10,5])

axs[0].set_title('Residuals Histogram')
axs[0].grid()
axs[0].hist(y - y_pred, alpha=0.5, color='C1', label='ga')
axs[0].hist(y - fit, alpha=0.5, color='C2', label='polyfit')
axs[0].set_xlabel('Frequency')
axs[0].set_ylabel('Residuals')
axs[0].legend()

axs[1].set_title('Residuals Plot')
axs[1].grid()
axs[1].plot(x, y - y_pred, alpha=0.5, color='C1', label='ga')
axs[1].plot(x, y - fit, alpha=0.5, color='C2', label='polyfit')
axs[1].set_xlabel('X')
axs[1].set_ylabel('Residuals')
axs[1].legend()

In [ ]:
print (f'\t ga\t\t\t polyfit')
print (f'weight\t {w_median}\t {z[0]}')
print (f'bias\t {b_median}\t {z[1]}')
print (f'mse\t {np.mean(np.square(y_pred - y))}\t {np.mean(np.square(fit - y))}')
print (f'mae\t {np.mean(np.abs(y_pred - y))}\t {np.mean(np.abs(fit - y))}')